In [2]:
from google.cloud import bigquery
client = bigquery.Client()
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")
dataset = client.get_dataset(dataset_ref)

C:\Users\ADMIN\AppData\Roaming\Python\Python312\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [4]:
tables = list(client.list_tables(dataset))
for table in tables:
    print(table.table_id)

taxi_trips


In [5]:
table_ref = dataset_ref.table("taxi_trips")
table = client.get_table(table_ref)
client.list_rows(table, max_results=5).to_dataframe()

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,5003bdd51918a9b5a52134096663b4d7e02395c5,2130bc5fd239a4e3b304662424fb4cc7db0ca7abf78cc5...,2013-02-25 14:15:00+00:00,2013-02-25 14:15:00+00:00,120,0.00,<NA>,<NA>,<NA>,<NA>,...,NaN,NaN,Cash,None,NaN,NaN,None,NaN,NaN,None
1,720534d264001b2644f682755b294067fdf1da21,4bb55b69e710d1792f5fd4888001f4ff0ce34040f976f8...,2013-06-02 08:15:00+00:00,2013-06-02 08:15:00+00:00,60,0.02,<NA>,<NA>,<NA>,<NA>,...,NaN,NaN,Cash,None,NaN,NaN,None,NaN,NaN,None
2,515b9a6d5234a4d1fb559dca27b3e1cc541a342d,4bb55b69e710d1792f5fd4888001f4ff0ce34040f976f8...,2013-06-24 06:15:00+00:00,2013-06-24 06:15:00+00:00,60,0.04,<NA>,<NA>,1,77,...,NaN,NaN,Cash,None,42.009623,-87.670167,POINT (-87.67016685690001 42.0096228806),41.986712,-87.663416,POINT (-87.6634164054 41.9867117999)
3,8afd48d966bf464e93127f43b72a8e405596a8eb,0150188f8c8e8973a198d4af0b427fca8ff48df2662d6b...,2013-02-01 07:00:00+00:00,2013-02-02 00:00:00+00:00,61560,0.00,<NA>,<NA>,<NA>,<NA>,...,0.0,0.0,Cash,None,NaN,NaN,None,NaN,NaN,None
4,fe73a44f01fdd2a3c740a01f53e97ec2ced93455,0150188f8c8e8973a198d4af0b427fca8ff48df2662d6b...,2013-02-01 07:00:00+00:00,2013-02-02 00:00:00+00:00,60900,0.00,<NA>,<NA>,<NA>,<NA>,...,0.0,0.0,Cash,None,NaN,NaN,None,NaN,NaN,None


In [8]:
query = """
    SELECT EXTRACT(year FROM trip_start_timestamp) as year, COUNT(1) as num_trips
    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
    GROUP BY year
    ORDER by year DESC
"""
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config=safe_config)
year_counts = query_job.to_dataframe()
year_counts

,year,num_trips
0,2023,6495415
1,2022,6382071
2,2021,3947677
3,2020,3888831
4,2019,16476440
5,2018,20731105
6,2017,24979611
7,2016,31756403
8,2015,32385527
9,2014,37395079


In [10]:
query = """
    WITH speed_table AS (
        SELECT EXTRACT(hour FROM trip_start_timestamp) as hour_of_day, trip_miles, trip_seconds
        FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
        WHERE trip_seconds > 0 AND
            trip_miles > 0 AND
            trip_start_timestamp > '2016-01-01' AND
            trip_start_timestamp < '2016-04-01'
    )
    SELECT hour_of_day, 
        COUNT(hour_of_day) as num_trips, 
        SUM(trip_miles) / (SUM(trip_seconds) / 3600) as avg_mph
    FROM speed_table
    GROUP BY hour_of_day
    ORDER BY hour_of_day
"""
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config=safe_config)
speeds_result = query_job.to_dataframe()
speeds_result

,hour_of_day,num_trips,avg_mph
0,0,203092,20.191744
1,1,178046,18.628598
2,2,143447,18.444370
3,3,108899,19.273107
4,4,80067,27.599669
5,5,75786,33.065604
6,6,102254,28.533112
7,7,187585,19.884592
8,8,284223,16.787900
9,9,306854,18.434124
